In [ ]:
!pip install numpy==1.26.0 scipy==1.14.0 meshio==5.3.5 polyscope==2.2.1 pbatoolkit

In [ ]:
import pbatoolkit as pbat
import numpy as np
import scipy as sp
import polyscope as ps
import polyscope.imgui as imgui
import time
import meshio
import os

In [ ]:
def signal(w: float, v: np.ndarray, t: float, c: float, k: float):
    u = c*np.sin(k*w*t)*v
    return u

### Define the parameters directly

In [ ]:
current_path = os.getcwd()

input_path = os.path.join(current_path, 'python' ,'examples', 'notebooks', 'resources', 'tetreahedron.obj')

mass_density = 1000.0
young_modulus = 1e6
poisson_ratio = 0.45
num_modes = 30

### Load the mesh and compute necessary quantities

In [ ]:
imesh = meshio.read(input_path)
V, C = imesh.points, imesh.cells_dict['tetra']
mesh = pbat.fem.Mesh(
    V.T, C.T, element=pbat.fem.Element.Tetrahedron, order=1)
x = mesh.X.reshape(mesh.X.shape[0]*mesh.X.shape[1], order='f')
detJeM = pbat.fem.jacobian_determinants(mesh, quadrature_order=2)
M = pbat.fem.MassMatrix(mesh, detJeM, rho=mass_density,
                        dims=3, quadrature_order=2).to_matrix()

detJeU = pbat.fem.jacobian_determinants(mesh, quadrature_order=1)
GNeU = pbat.fem.shape_function_gradients(mesh, quadrature_order=1)
Y = np.full(mesh.E.shape[1], young_modulus)
nu = np.full(mesh.E.shape[1], poisson_ratio)
hep = pbat.fem.HyperElasticPotential(
    mesh, detJeU, GNeU, Y, nu, energy=pbat.fem.HyperElasticEnergy.StableNeoHookean, quadrature_order=1)
hep.precompute_hessian_sparsity()
hep.compute_element_elasticity(x)
U, gradU, HU = hep.eval(), hep.gradient(), hep.hessian()
sigma = -1e-5
leigs, Veigs = sp.sparse.linalg.eigsh(HU, k=num_modes, M=M, sigma=sigma, which='LM')
Veigs = Veigs / sp.linalg.norm(Veigs, axis=0, keepdims=True)
leigs[leigs <= 0] = 0
w = np.sqrt(leigs)

### Setup Polyscope and visualization

In [ ]:
ps.set_up_dir('z_up')
ps.set_front_dir('neg_y_front')
ps.set_ground_plane_mode('shadow_only')
ps.init()
vm = ps.register_volume_mesh('model', mesh.X.T, mesh.E.T)
mode = 6
t0 = time.time()
t = 0
c = 0.15
k = 0.05

def callback():
    global mode, c, k
    changed, mode = imgui.InputInt('Mode', mode)
    changed, c = imgui.InputFloat('Wave amplitude', c)
    changed, k = imgui.InputFloat('Wave frequency', k)

    t = time.time() - t0
    X = mesh.X.T + signal(w[mode], Veigs[:, mode],
                          t, c, k).reshape(mesh.X.shape[1], 3)
    vm.update_vertex_positions(X)

ps.set_user_callback(callback)
ps.show()